In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/ViT5/examples


In [2]:
import sys
sys.path.append("")

In [3]:
# !wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
# !unzip 'master.zip'

In [4]:
import glob
import pandas as pd
pd.set_option('display.max_columns', None)
import concurrent.futures
from datasets import *
import datasets
from sklearn.model_selection import train_test_split
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# from vncorenlp import VnCoreNLP
import torch 
import nltk
from nltk.tokenize import sent_tokenize



/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
OUTPUT_DIR = 'vietnamese_mt5_summary_model_2'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/mrzaizai2k/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Processing data

In [7]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('data/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('data/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('data/vietnews-master/data/test_tokenized/*')

In [8]:
def read_content(pathfile):
    """
    Input: Path of txt file
    Output: A dictionary has keys 'original' and 'summary'
    """
    with open(pathfile) as f:
      rows  = f.readlines()
      original = ' '.join(''.join(rows[4:]).split('\n'))
      summary = ' '.join(rows[2].split('\n'))
            
    return {'file' : pathfile,
              'original': original, 
              'summary': summary}

In [9]:
read_content(train_paths[0])

{'file': 'data/vietnews-master/data/train_tokenized/006157.txt.seg',
 'original': 'Tập 4 Thần_tượng Âm_nhạc nhí - Vietnam_Idol_Kids 2017 lên sóng tối 2/6 với màn tranh tài của top 8 thí_sinh nữ để lựa_chọn ra 5 tấm vé vào tiếp vòng trong . Trong đó , 3 giám_khảo Isaac - Văn_Mai_Hương - Bích_Phương được quyền lựa_chọn 3 thí_sinh . 2 thí_sinh còn lại sẽ do các giám_khảo khách mời quyết_định . Nữ ca_sĩ Văn_Mai_Hương xúc_động : “ Cô có may_mắn năm nay ngồi ở vị_trí ban giám_khảo , may_mắn hơn là được gặp con . Mỗi khi gặp Hiền , cô tự thấy bản_thân cô rất kém , bởi có lúc cô không trân_trọng cũng như không tin vào bản_thân ... Cảm_ơn con , bởi đôi_khi có những cái cô không bằng con được , đó là sự lạc_quan . Và cô tin , còn rất nhiều người phải học đức_tính lạc_quan này của Hiền . Con hát rất là hay ” . Đồng_tình với ý_kiến của đồng_nghiệp , Bích_Phương cũng xúc_động chia_sẻ : “ Giọt nước_mắt dành cho con là sự khâm_phục chứ không phải là thương_cảm . Cô chưa bao_giờ thấy con buồn . Từ lúc

In [10]:
def get_dataframe(pathfiles):
    with concurrent.futures.ProcessPoolExecutor() as executor:
      data = executor.map(read_content, pathfiles)
    
    # Make blank dataframe
    data_df = list()
    for d in data:
      data_df.append(d)
    data_df = pd.DataFrame(data_df)
    data_df.dropna(inplace = True)
    data_df = data_df.sample(frac=1).reset_index(drop=True)

    return data_df

In [11]:
train_df = get_dataframe(train_paths[10001:15001])

In [12]:
val_df = get_dataframe(val_paths[:500])
test_df = get_dataframe(test_paths[:100])

In [13]:
test_df.head()

,file,original,summary
0,data/vietnews-master/data/test_tokenized/00682...,"Tờ al - Watan hôm 14/3 cho_hay , các máy_bay c...",Các đơn_vị không_quân Syria và Nga đã tiến_hàn...
1,data/vietnews-master/data/test_tokenized/01857...,Đây là dự_án được các nhà_đầu_tư cũng như các ...,Công_ty_Cổ_phần Vinhomes vừa chính_thức ra_mắt...
2,data/vietnews-master/data/test_tokenized/01472...,"Theo kế_hoạch của UBND thành_phố , tại lễ khai...","Ông Nguyễn_Đức_Chung , Chủ_tịch UBND TP Hà_Nội..."
3,data/vietnews-master/data/test_tokenized/01308...,Tổng_Công_ty Đầu_tư phát_triển nhà và đô_thị (...,Tổng_công_ty HUD vừa đề_xuất với Bộ Xây_dựng c...
4,data/vietnews-master/data/test_tokenized/01895...,Hoạ_sĩ Hà_Hùng_Dũng theo_đuổi niềm đam_mê khai...,"Vừa mang tranh lên Sa_Pa triển_lãm , hoạ_sĩ Hà..."


In [14]:
# test_df.to_parquet("data/vietnews/test.parquet", index=False)

In [15]:
# train_df = pd.read_parquet("data/vietnews/train.parquet")
# val_df = pd.read_parquet("data/vietnews/val.parquet")
# test_df = pd.read_parquet("data/vietnews/test.parquet")
# test_df.head()

In [16]:
# import matplotlib.pyplot as plt

# train_df['word_count'] = train_df['summary'].apply(lambda x: len(x.split()))

# # Plot the histogram
# plt.figure(figsize=(10, 6))
# plt.hist(train_df['word_count'], bins=100, edgecolor='black')
# plt.title('Histogram of Number of Words in Sentences')
# plt.xlabel('Number of Words')
# plt.ylabel('Frequency')
# plt.show()

## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [17]:
# pretrained_model = "google/mt5-small"
pretrained_model = "vietnamese_mt5_summary_model"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)


In [18]:
train_data =  Dataset.from_pandas(train_df)
val_data =  Dataset.from_pandas(val_df)
test_data =  Dataset.from_pandas(test_df)

In [19]:
batch_size=16  # change to 16 for full training

max_input_length = 512 #1024
max_target_length = 128 #128


def process_data_to_model_inputs(batch):
    model_inputs = tokenizer(
        batch["original"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        batch["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
# train_data = train_data.select(range(32))

train_data_batch = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file","original", "summary"],
)
train_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
# val_data = val_data.select(range(16))

val_data_batch = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["file", "original", "summary"],
)
val_data_batch.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)

Map: 100%|██████████| 500/500 [00:00<00:00, 1656.33 examples/s]


In [20]:
tokenizer.pad_token_id

0

In [21]:
train_df.iloc[0]

file        data/vietnews-master/data/train_tokenized/0574...
original    Hình_ảnh người đàn_ông Trung_Quốc giương cung ...
summary     Một người đàn_ông ở tỉnh Phúc_Kiến , Trung_Quố...
Name: 0, dtype: object

In [22]:
a = tokenizer(str(train_df["summary"].iloc[0]))
print(str(train_df["summary"].iloc[0]))
a

Một người đàn_ông ở tỉnh Phúc_Kiến , Trung_Quốc dùng cung tên kết_liễu mạng sống của chó cảnh nhà hàng_xóm . 


{'input_ids': [352, 1266, 1672, 355, 2353, 290, 1735, 259, 1269, 259, 270, 4994, 1265, 2879, 5327, 290, 557, 10513, 259, 261, 259, 9978, 290, 1290, 8594, 331, 2201, 259, 13220, 259, 10282, 408, 1878, 290, 494, 6985, 273, 326, 1292, 318, 259, 263, 2160, 317, 708, 262, 259, 55337, 317, 5079, 677, 369, 3057, 290, 329, 18768, 259, 260, 259, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
# tokenizer.convert_ids_to_tokens(train_data_batch[0]["labels"])

In [24]:
# train_data_batch[0]

### **Warm-starting the Encoder-Decoder Model**

In [25]:
from transformers import AutoModelForSeq2SeqLM, GenerationConfig

model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model).to(device)
my_config = model.generation_config
print("generation_config", model.generation_config)

my_config.max_length = 128
my_config.length_penalty = 2.0
my_config.num_beams= 4
my_config.no_repeat_ngram_size= 3
my_config.early_stopping = True
print("generation_config", model.generation_config)


generation_config GenerationConfig {
  "decoder_start_token_id": 0,
  "early_stopping": true,
  "eos_token_id": 1,
  "length_penalty": 2.0,
  "max_length": 256,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 0
}

generation_config GenerationConfig {
  "decoder_start_token_id": 0,
  "early_stopping": true,
  "eos_token_id": 1,
  "length_penalty": 2.0,
  "max_length": 128,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 0
}



### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [26]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

2024-06-10 15:34:24.260710: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 15:34:24.287332: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 15:34:24.831075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [27]:
import numpy as np
import evaluate

rouge_score = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [28]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='pt')

In [29]:
features = [train_data_batch[i] for i in range(2)]
data_collator(features)

/home/mrzaizai2k/code_Bao/ViT5/venv/lib/python3.10/site-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


{'input_ids': tensor([[ 447, 1253,  290,  ...,    0,    0,    0],
        [ 320, 1833,  313,  ...,  268,  290,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  352,  1266,  1672,   355,  2353,   290,  1735,   259,  1269,   259,
           270,  4994,  1265,  2879,  5327,   290,   557, 10513,   259,   261,
           259,  9978,   290,  1290,  8594,   331,  2201,   259, 13220,   259,
         10282,   408,  1878,   290,   494,  6985,   273,   326,  1292,   318,
           259,   263,  2160,   317,   708,   262,   259, 55337,   317,  5079,
           677,   369,  3057,   290,   329, 18768,   259,   260,   259,     1,
          -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [  320,  1833,   408,   924,   317, 11825,   819,  1534,   977,  5620,
           290, 26565,   334,   355,  3921, 14134,   977,  1690,   259,  1369,
          1516,  1302,   290,  2535,  1405,  2052,   885,   259,  9762,   290,
           2

Cool! Finally, we start training.

In [30]:
# set training arguments - these params are not really tuned, feel free to change
logging_steps = min(2000,len(train_data_batch) // batch_size)
print("logging_steps", logging_steps)

training_args = Seq2SeqTrainingArguments(
    output_dir= OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    eval_strategy="epoch",
    learning_rate=1e-3,
    # do_train=True,
    # do_eval=True,
    logging_steps=logging_steps,  # set to 2000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=750,  # set to 7500 for full training
    warmup_steps=3000,  # set to 3000 for full training
    num_train_epochs=10, #uncomment for full training
    overwrite_output_dir=True,
    optim='adafactor',
    save_total_limit=3,
    # load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model,
    args=training_args,
    train_dataset=train_data_batch,
    eval_dataset=val_data_batch,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

logging_steps 312


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.575700,1.422527,57.382300,23.843700,35.628800,37.107900
2,1.545200,1.421908,56.809000,23.465700,35.097700,36.760600
3,1.507400,1.412181,55.909500,23.012800,35.027400,36.444700
4,1.470100,1.425617,56.621000,23.187600,35.132300,36.551800
5,1.431000,1.438057,57.206700,23.608700,35.123900,36.742100
6,1.392900,1.433783,57.124800,23.944600,35.366600,36.997400
7,1.355800,1.472713,57.048200,23.100100,34.818700,36.181700
8,1.319700,1.492798,56.040900,23.170200,35.441400,36.858000
9,1.286100,1.491692,57.141600,23.755500,35.574700,36.941800
10,1.236700,1.481348,57.261800,23.556200,35.471700,37.125900


TrainOutput(global_step=3130, training_loss=1.4115332551657582, metrics={'train_runtime': 18121.2358, 'train_samples_per_second': 2.759, 'train_steps_per_second': 0.173, 'total_flos': 2.6437484544e+16, 'train_loss': 1.4115332551657582, 'epoch': 10.0})

In [31]:
trainer.save_model(OUTPUT_DIR)

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]
events.out.tfevents.1718008467.DESKTOP-H2CRQMR.907286.0: 100%|██████████| 12.4k/12.4k [00:01<00:00, 10.3kB/s]
model.safetensors: 100%|██████████| 1.20G/1.20G [01:04<00:00, 18.6MB/s]
Upload 2 LFS files: 100%|██████████| 2/2 [01:05<00:00, 32.51s/it]


In [32]:
# !gsutil -m cp -r '/content/training/*' 'gs://kaggle-vbdi-test/training_Data'

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [38]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, use_fast=False)

# model = EncoderDecoderModel.from_pretrained(OUTPUT_DIR + "/checkpoint-4000")
model = AutoModelForSeq2SeqLM.from_pretrained(OUTPUT_DIR, torch_dtype=torch.bfloat16,).to(device)
print("generation_config", model.generation_config)


# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    outputs = model.generate(input_ids, attention_mask=attention_mask,
                            #  max_length=256,
                        # num_beams=4,
                        # no_repeat_ngram_size=3,
                        # early_stopping=True,
                        )

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

generation_config GenerationConfig {
  "decoder_start_token_id": 0,
  "early_stopping": true,
  "eos_token_id": 1,
  "length_penalty": 2.0,
  "max_length": 128,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 0
}



Map: 100%|██████████| 100/100 [00:53<00:00,  1.87 examples/s]


In [39]:
rouge_output = rouge_score.compute(predictions=pred_str, references=label_str, use_stemmer=True)
rouge_output = {key: value * 100 for key, value in rouge_output.items()}
rouge_output

{'rouge1': 58.415897565315625,
 'rouge2': 24.991767036179702,
 'rougeL': 36.06861271388169,
 'rougeLsum': 36.07599070665035}

In [40]:
for i in range(10):
    print('\nPrediction: ',pred_str[i])
    print('Truth: ',label_str[i])
    print('Content: ',test_data[i]['original'])
    


Prediction:  Các máy_bay chiến_đấu Nga và Syria đã tiến_hành các cuộc không_kích nhằm vào nơi ẩn_náu của các tay súng khủng_bố ở miền Nam và Đông_Idlib bằng tên_lửa hạng nặng.
Truth:  Các đơn_vị không_quân Syria và Nga đã tiến_hành tấn_công mạnh_mẽ nhằm vào cứ_điểm của các phiến quân ở Bắc_Hama và Nam_Idlib để đối_phó với các đòn đánh của nhóm Tahrir al - Sham al - Hay ' at ( hay Al - Nusra ) tại khu_vực phi quân_sự . 
Content:  Tờ al - Watan hôm 14/3 cho_hay , các máy_bay chiến_đấu Nga và Syria đã tiến_hành các cuộc không_kích nhằm vào nơi ẩn_náu của các tay súng khủng_bố ở al - Tamane ' ah , al - Nayrab , Saraqib , Kafr_Amim và Ma ' arat Hormat ở miền Nam và Đông_Idlib cũng như cứ_điểm của chúng ở Bắc và Tây_Bắc_Hama Các nguồn_tin thực_địa mô_tả vụ tấn_công hôm 13/3 là cuộc không_kích có mức_độ huỷ_diệt lớn nhất của các đơn_vị không_quân Nga và Syria kể từ tháng 9/2018 , khiến số_lượng lớn khủng_bố thiệt_mạng và bị_thương . Cùng với các cuộc không_kích , lực_lượng quân_đội Syria đã 

In [41]:
test_data[0]['file']

'data/vietnews-master/data/test_tokenized/006820.txt.seg'